In [6]:
import getpass

db_user = getpass.getpass()
db_password = getpass.getpass()

 ·········
 ········


In [7]:
from sqlalchemy import create_engine
import datetime as dt
import pandas as pd

mysql_engine = create_engine('mysql+pymysql://{}:{}@analquick.cogfzn9i6pkm.ap-northeast-2.rds.amazonaws.com:3306/quicket_log'.format(db_user, db_password))
mysql = mysql_engine.connect()

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'analquick.cogfzn9i6pkm.ap-northeast-2.rds.amazonaws.com' (timed out)")
(Background on this error at: http://sqlalche.me/e/e3q8)

In [9]:
sql = '''
SELECT distinct channel_id
FROM tb_message
WHERE update_time >= '2019-11-01 23:20:00' AND update_time < '2019-11-01 23:30:00'
'''
stat = pd.read_sql(sql, mysql)
stat.head()

,channel_id
0,131128739
1,164513504
2,165935146
3,166159440
4,166815805


In [12]:
import datetime
INTERVAL_MIN = 30
start = datetime.datetime(2019, 11, 1, 0, 0, 0)
end = datetime.datetime(2019, 11, 2, 0, 0, 0)
interval = int((end-start).total_seconds()/60/INTERVAL_MIN)
interval

48

In [13]:
from multiprocessing import Pool
import tqdm

result = []

def fetch(dates):
    db = mysql_engine.connect()
    start = dates[0]
    end = dates[1]
    now = datetime.datetime.now()
    sql = '''
    SELECT distinct channel_id
    FROM tb_message
    WHERE update_time >= '{}' AND update_time < '{}'
    '''.format(str(start), str(end))
    df = pd.read_sql(sql, db)
    #print("End: {}, {}, {} seconds".format(start, end, (datetime.datetime.now() - now).total_seconds()))
    db.close()
    return df['channel_id'].values.tolist()

date_ranges = []
for i in range(interval):
    dt1 = start + datetime.timedelta(seconds=60*i*INTERVAL_MIN)
    dt2 = start + datetime.timedelta(seconds=60*(i+1)*INTERVAL_MIN)
    date_ranges.append([dt1, dt2])

pool = Pool(processes = 30)

values = []
for value in tqdm.tqdm(pool.imap_unordered(fetch, date_ranges), total=len(date_ranges)):
    values.append(value)

pool.close()
pool.join()


100%|██████████| 48/48 [00:19<00:00,  2.50it/s]


In [14]:
channel_ids = set()
for value in values:
    channel_ids.update(value)
len(channel_ids)

182446